In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_csv("train.csv")
df.head()

,id_viaje,id_usuario,id_vehicle,inicio_viaje,final_viaje,lat_ini,long_ini,lat_fin,long_fin,cluster
0,11544264,14999,13184,2022-01-01T00:40:29Z,2022-01-01T01:06:06Z,40.439560,-3.710959,40.531506,-3.636610,73
1,11544318,700120,13230,2022-01-01T00:42:58Z,2022-01-01T01:01:20Z,40.420307,-3.695659,40.442307,-3.704213,8
2,11544295,697271,9893,2022-01-01T00:43:47Z,2022-01-01T01:01:22Z,40.428822,-3.703689,40.408131,-3.706030,40
3,11544349,524536,9998,2022-01-01T00:45:50Z,2022-01-01T01:09:42Z,40.425667,-3.689260,40.414787,-3.701125,7
4,11544356,697386,9789,2022-01-01T00:46:19Z,2022-01-01T01:22:58Z,40.421139,-3.696565,40.421120,-3.696330,8


In [3]:
# Convert 'inicio_viaje' column to datetime format
df['date2'] = pd.to_datetime(df['inicio_viaje'])

# Extracr the date, hour, day of the week and month from the 'date2' column
df['date'] = df['date2'].dt.date
df['hour'] = pd.DatetimeIndex(df['date2']).hour
df['dia_semana'] = df['date2'].dt.dayofweek
df['mes'] = df['date2'].dt.month

# Define a function to determine the season of the year based on the month
def get_season(mes):
    if 3 <= mes <= 5:
        return '1'  # Spring
    elif 6 <= mes <= 8:
        return '2'  # Summer
    elif 9 <= mes <= 11:
        return '3'  # Autumn
    else:
        return '4'  # Winter

# Aplly the 'get_season' function to create a new column 'season'
df['season'] = df['mes'].apply(get_season)

# Show the first rows of the DataFrame with the new agegated columns
df.head()


,id_viaje,id_usuario,id_vehicle,inicio_viaje,final_viaje,lat_ini,long_ini,lat_fin,long_fin,cluster,date2,date,hour,dia_semana,mes,season
0,11544264,14999,13184,2022-01-01T00:40:29Z,2022-01-01T01:06:06Z,40.439560,-3.710959,40.531506,-3.636610,73,2022-01-01 00:40:29+00:00,2022-01-01,0,5,1,4
1,11544318,700120,13230,2022-01-01T00:42:58Z,2022-01-01T01:01:20Z,40.420307,-3.695659,40.442307,-3.704213,8,2022-01-01 00:42:58+00:00,2022-01-01,0,5,1,4
2,11544295,697271,9893,2022-01-01T00:43:47Z,2022-01-01T01:01:22Z,40.428822,-3.703689,40.408131,-3.706030,40,2022-01-01 00:43:47+00:00,2022-01-01,0,5,1,4
3,11544349,524536,9998,2022-01-01T00:45:50Z,2022-01-01T01:09:42Z,40.425667,-3.689260,40.414787,-3.701125,7,2022-01-01 00:45:50+00:00,2022-01-01,0,5,1,4
4,11544356,697386,9789,2022-01-01T00:46:19Z,2022-01-01T01:22:58Z,40.421139,-3.696565,40.421120,-3.696330,8,2022-01-01 00:46:19+00:00,2022-01-01,0,5,1,4


In [4]:
# Group the dataframe by different columns and count the number of trips
num_viajes=df.groupby(['cluster', 'hour','dia_semana','mes','season','date'])['cluster'].count().to_frame(name = 'trips').reset_index()
num_viajes.head()

,cluster,hour,dia_semana,mes,season,date,trips
0,1,0,0,2,4,2022-02-14,1
1,1,0,0,2,4,2022-02-21,1
2,1,0,0,3,1,2022-03-28,1
3,1,0,0,4,1,2022-04-25,2
4,1,0,0,5,1,2022-05-02,1


In [5]:
# Convierte las columnas específicas a tipos de datos de cadena (str)
num_viajes['cluster'] = num_viajes['cluster'].astype(str)
num_viajes['hour'] = num_viajes['hour'].astype(str)
num_viajes['dia_semana'] = num_viajes['dia_semana'].astype(str)
num_viajes['season'] = num_viajes['season'].astype(str)
num_viajes['date'] = num_viajes['date'].astype(str)
num_viajes['mes'] = num_viajes['mes'].astype(str)



In [6]:
num_viajes.dtypes

cluster       object
hour          object
dia_semana    object
mes           object
season        object
date          object
trips          int64
dtype: object

**Transformation of sample submission to test (real predictions)**

In [7]:
test = pd.read_csv("sample_submission.csv")

In [8]:
#Dvide the 'Id' column using the character "_" as separator and expand the results into separate columns
split_test= test["Id"].str.split("_", n = 3, expand = True)

In [9]:
#Assign the resulting columns from the split to the 'test' DataFrame
test['cluster'] = split_test[0]
test['date'] = split_test[1]
test['hour'] = split_test[2]

#Convert the 'date' column to datetime format
test['date'] = pd.to_datetime(test['date'])

#Extract the day of the week, month and season from the 'date' column
test['dia_semana'] = test['date'].dt.dayofweek
test['mes'] = test['date'].dt.month
test['season'] = test['mes'].apply(get_season)

# Eliminate trips column if exists
test = test.drop(columns=['trips'])


In [10]:
#Convert the specific columns to string data types (str)
test['date']=test['date'].astype(str)
test['dia_semana']=test['dia_semana'].astype(str)
test['mes']=test['mes'].astype(str)

CÁLCULO DE MEDIA POR CLUSTER 

In [11]:
#Calculate the mean number of trips per cluster and hour
media_por_cluster = num_viajes.groupby(['cluster', 'hour'])['trips'].mean().to_frame(name='trips').reset_index()

In [12]:
media_por_cluster['hour'] = media_por_cluster['hour'].astype(object)

In [13]:
# Mix the DataFrames 'test' and 'media_por_cluster' by the columns 'cluster' and 'hour'
test_media = pd.merge(test, media_por_cluster, how='left', left_on=['cluster', 'hour'], right_on=['cluster', 'hour'])

#Fill null values in the 'trips' column with zeros
test_media['trips'] = test_media['trips'].fillna(0)


In [14]:
test_media['hour'] = test_media['hour'].astype(int)

In [15]:
num_viajes['hour'] = num_viajes['hour'].astype(int)

**WEATHER MERGING**

In [41]:
weather = pd.read_csv('weather.csv')

In [18]:
# Create a list of hours from 0 to 23
horas = list(range(24))

# Repeat each row in df2 for each hour and assign the corresponding hour
weather_expandido = pd.DataFrame()
for _, fila in weather.iterrows():
    fila_repetida = pd.DataFrame([fila] * 24)
    fila_repetida['hour'] = horas
    weather_expandido = pd.concat([weather_expandido, fila_repetida], ignore_index=True)
weather_expandido

,date,tavg,tmin,tmax,prcp,wdir,wspd,pres,hour
0,2022-01-01,-0.794859,-0.940391,-0.580302,-0.341636,-1.19286,-1.129823,1.649661,0
1,2022-01-01,-0.794859,-0.940391,-0.580302,-0.341636,-1.19286,-1.129823,1.649661,1
2,2022-01-01,-0.794859,-0.940391,-0.580302,-0.341636,-1.19286,-1.129823,1.649661,2
3,2022-01-01,-0.794859,-0.940391,-0.580302,-0.341636,-1.19286,-1.129823,1.649661,3
4,2022-01-01,-0.794859,-0.940391,-0.580302,-0.341636,-1.19286,-1.129823,1.649661,4
...,...,...,...,...,...,...,...,...,...
8755,2022-12-31,-1.117715,-1.056661,-0.968687,-0.341636,-0.39794,-1.526017,1.395076,19
8756,2022-12-31,-1.117715,-1.056661,-0.968687,-0.341636,-0.39794,-1.526017,1.395076,20
8757,2022-12-31,-1.117715,-1.056661,-0.968687,-0.341636,-0.39794,-1.526017,1.395076,21
8758,2022-12-31,-1.117715,-1.056661,-0.968687,-0.341636,-0.39794,-1.526017,1.395076,22


In [19]:
#Create a new row with the date of January 1, 2022 and the data of December 31, 2022
new_row = weather_expandido.iloc[-1].copy()
new_row['date'] = '2023-01-01'
new_row_df = pd.DataFrame([new_row])

#Add the new row to the DataFrame
weather_expandido = pd.concat([weather_expandido, new_row_df],ignore_index = True)
weather_expandido['hour'].iloc[8760] = 0

#Show the last rows of the updated DataFrame to verify the addition
weather_expandido.tail()

C:\Users\alvar\AppData\Local\Temp\ipykernel_1844\1595500720.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_expandido['hour'].iloc[8760] = 0


,date,tavg,tmin,tmax,prcp,wdir,wspd,pres,hour
8756,2022-12-31,-1.117715,-1.056661,-0.968687,-0.341636,-0.39794,-1.526017,1.395076,20
8757,2022-12-31,-1.117715,-1.056661,-0.968687,-0.341636,-0.39794,-1.526017,1.395076,21
8758,2022-12-31,-1.117715,-1.056661,-0.968687,-0.341636,-0.39794,-1.526017,1.395076,22
8759,2022-12-31,-1.117715,-1.056661,-0.968687,-0.341636,-0.39794,-1.526017,1.395076,23
8760,2023-01-01,-1.117715,-1.056661,-0.968687,-0.341636,-0.39794,-1.526017,1.395076,0


In [20]:
weather_expandido['date']

0       2022-01-01
1       2022-01-01
2       2022-01-01
3       2022-01-01
4       2022-01-01
           ...    
8756    2022-12-31
8757    2022-12-31
8758    2022-12-31
8759    2022-12-31
8760    2023-01-01
Name: date, Length: 8761, dtype: object

In [21]:
weather_train = weather_expandido[weather_expandido['date'] <= '2022-09-30']
weather_test = weather_expandido[weather_expandido['date'] >= '2022-09-30']

In [22]:
weather_test

,date,tavg,tmin,tmax,prcp,wdir,wspd,pres,hour
6528,2022-09-30,-0.285740,-0.388110,-0.245859,-0.315915,2.005748,-0.126131,-0.466580,0
6529,2022-09-30,-0.285740,-0.388110,-0.245859,-0.315915,2.005748,-0.126131,-0.466580,1
6530,2022-09-30,-0.285740,-0.388110,-0.245859,-0.315915,2.005748,-0.126131,-0.466580,2
6531,2022-09-30,-0.285740,-0.388110,-0.245859,-0.315915,2.005748,-0.126131,-0.466580,3
6532,2022-09-30,-0.285740,-0.388110,-0.245859,-0.315915,2.005748,-0.126131,-0.466580,4
...,...,...,...,...,...,...,...,...,...
8756,2022-12-31,-1.117715,-1.056661,-0.968687,-0.341636,-0.397940,-1.526017,1.395076,20
8757,2022-12-31,-1.117715,-1.056661,-0.968687,-0.341636,-0.397940,-1.526017,1.395076,21
8758,2022-12-31,-1.117715,-1.056661,-0.968687,-0.341636,-0.397940,-1.526017,1.395076,22
8759,2022-12-31,-1.117715,-1.056661,-0.968687,-0.341636,-0.397940,-1.526017,1.395076,23


In [23]:
num_viajes_weather = pd.merge(num_viajes,weather_train, on = ['date','hour'])
num_viajes_weather.sort_values(['date','hour'])

,cluster,hour,dia_semana,mes,season,date,trips,tavg,tmin,tmax,prcp,wdir,wspd,pres
363863,3,0,5,1,4,2022-01-01,1,-0.794859,-0.940391,-0.580302,-0.341636,-1.192860,-1.129823,1.649661
363864,5,0,5,1,4,2022-01-01,1,-0.794859,-0.940391,-0.580302,-0.341636,-1.192860,-1.129823,1.649661
363865,7,0,5,1,4,2022-01-01,1,-0.794859,-0.940391,-0.580302,-0.341636,-1.192860,-1.129823,1.649661
363866,8,0,5,1,4,2022-01-01,2,-0.794859,-0.940391,-0.580302,-0.341636,-1.192860,-1.129823,1.649661
363867,10,0,5,1,4,2022-01-01,1,-0.794859,-0.940391,-0.580302,-0.341636,-1.192860,-1.129823,1.649661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287390,95,21,4,9,3,2022-09-30,1,-0.285740,-0.388110,-0.245859,-0.315915,2.005748,-0.126131,-0.466580
287391,97,21,4,9,3,2022-09-30,5,-0.285740,-0.388110,-0.245859,-0.315915,2.005748,-0.126131,-0.466580
287392,98,21,4,9,3,2022-09-30,9,-0.285740,-0.388110,-0.245859,-0.315915,2.005748,-0.126131,-0.466580
287393,99,21,4,9,3,2022-09-30,4,-0.285740,-0.388110,-0.245859,-0.315915,2.005748,-0.126131,-0.466580


In [24]:
test['hour'] = test['hour'].astype(int)

In [25]:
test = pd.merge(test,weather_test, on = ['date','hour'])
test.sort_values(['date','hour'])

,Id,cluster,date,hour,dia_semana,mes,season,tavg,tmin,tmax,prcp,wdir,wspd,pres
0,72_2022-09-30_22,72,2022-09-30,22,4,9,3,-0.285740,-0.388110,-0.245859,-0.315915,2.005748,-0.126131,-0.466580
1,93_2022-09-30_22,93,2022-09-30,22,4,9,3,-0.285740,-0.388110,-0.245859,-0.315915,2.005748,-0.126131,-0.466580
2,36_2022-09-30_22,36,2022-09-30,22,4,9,3,-0.285740,-0.388110,-0.245859,-0.315915,2.005748,-0.126131,-0.466580
3,83_2022-09-30_22,83,2022-09-30,22,4,9,3,-0.285740,-0.388110,-0.245859,-0.315915,2.005748,-0.126131,-0.466580
4,92_2022-09-30_22,92,2022-09-30,22,4,9,3,-0.285740,-0.388110,-0.245859,-0.315915,2.005748,-0.126131,-0.466580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131833,45_2023-01-01_00,45,2023-01-01,0,6,1,4,-1.117715,-1.056661,-0.968687,-0.341636,-0.397940,-1.526017,1.395076
131834,59_2023-01-01_00,59,2023-01-01,0,6,1,4,-1.117715,-1.056661,-0.968687,-0.341636,-0.397940,-1.526017,1.395076
131835,25_2023-01-01_00,25,2023-01-01,0,6,1,4,-1.117715,-1.056661,-0.968687,-0.341636,-0.397940,-1.526017,1.395076
131836,37_2023-01-01_00,37,2023-01-01,0,6,1,4,-1.117715,-1.056661,-0.968687,-0.341636,-0.397940,-1.526017,1.395076


**Split for metrics**

In [26]:
num_viajes_features = num_viajes_weather[['cluster', 'hour', 'dia_semana', 'mes',
       'tmin','pres']]
num_viajes_labels = num_viajes_weather['trips']

In [27]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(num_viajes_features, num_viajes_labels, test_size = 0.25, random_state = 42)

In [28]:
test_features.sort_values(['hour'])

,cluster,hour,dia_semana,mes,tmin,pres
363972,86,0,5,4,-0.998526,-2.471441
3676,21,0,3,4,-0.315441,0.249441
322620,35,0,3,5,0.992593,-0.307464
7818,85,0,5,5,0.803655,-0.291553
362852,64,0,1,3,-0.940391,-0.116525
...,...,...,...,...,...,...
308191,46,23,0,5,0.193239,0.185795
386264,90,23,5,1,-1.405470,2.477064
385892,59,23,2,4,-0.867722,-0.848458
316461,62,23,4,1,-1.216531,2.206567


**Train with split**

In [29]:
model = GradientBoostingRegressor(loss="absolute_error", n_estimators=100, learning_rate=0.2, max_depth=5,random_state=42)

model.fit(train_features, train_labels)

GradientBoostingRegressor(learning_rate=0.2, loss='absolute_error', max_depth=5,
                          random_state=42)

In [30]:
#Define hiperparameters to try
parametros = {
    'n_estimators': [50, 75, 100],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}

# Configure GridSearchCV
busqueda_grid = GridSearchCV(model, parametros, cv=3, scoring='neg_mean_squared_error', verbose=1)

# Execute the search
busqueda_grid.fit(train_features, train_labels)

# Best parameters
print("Best parameters:", busqueda_grid.best_params_)

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best parameters: {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 100}


In [31]:
y_pred = model.predict(test_features)

**Metrics with split**

In [32]:
test_features

,cluster,hour,dia_semana,mes,tmin,pres
284419,80,21,4,1,-1.216531,2.206567
385861,94,23,2,2,-0.780520,0.933640
160315,77,15,3,2,-1.201998,1.506457
214353,13,18,1,9,-0.228239,-0.323376
306587,35,22,6,5,0.571115,-1.755419
...,...,...,...,...,...,...
401047,46,2,4,2,-0.925857,0.090326
141551,69,14,3,5,0.701919,-0.100614
62580,72,9,3,2,-1.071194,0.774524
402133,87,5,3,2,-0.780520,0.392646


In [33]:
# Calculate the absolute errors
errors = abs(y_pred - test_labels)

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', (np.mean(errors)))

Mean Absolute Error: 1.277623474539798


**FEATURE IMPORTANCE**

In [ ]:
def plot_importance(model, features, num=len(num_viajes_features), save=False):
    feature_imp = pd.DataFrame({'Value': model.feature_importances_, 'Feature': num_viajes_features.columns})
    print(feature_imp.sort_values("Value",ascending=False))
    plt.figure(figsize=(10, 10))
    sns.set(font_scale=1)
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value",
                                                                     ascending=False)[0:num])
    plt.title('Features')
    plt.tight_layout()
    plt.show()
    if save:
        plt.savefig('importances.png')

plot_importance(model, train_features)

**Real prediction**

In [34]:
num_viajes_weather.sort_values('date')

,cluster,hour,dia_semana,mes,season,date,trips,tavg,tmin,tmax,prcp,wdir,wspd,pres
13899,24,1,5,1,4,2022-01-01,1,-0.794859,-0.940391,-0.580302,-0.341636,-1.192860,-1.129823,1.649661
381279,3,18,5,1,4,2022-01-01,1,-0.794859,-0.940391,-0.580302,-0.341636,-1.192860,-1.129823,1.649661
337165,98,10,5,1,4,2022-01-01,1,-0.794859,-0.940391,-0.580302,-0.341636,-1.192860,-1.129823,1.649661
337164,82,10,5,1,4,2022-01-01,1,-0.794859,-0.940391,-0.580302,-0.341636,-1.192860,-1.129823,1.649661
337163,81,10,5,1,4,2022-01-01,1,-0.794859,-0.940391,-0.580302,-0.341636,-1.192860,-1.129823,1.649661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184901,58,16,4,9,3,2022-09-30,6,-0.285740,-0.388110,-0.245859,-0.315915,2.005748,-0.126131,-0.466580
184902,59,16,4,9,3,2022-09-30,2,-0.285740,-0.388110,-0.245859,-0.315915,2.005748,-0.126131,-0.466580
184903,60,16,4,9,3,2022-09-30,8,-0.285740,-0.388110,-0.245859,-0.315915,2.005748,-0.126131,-0.466580
184877,32,16,4,9,3,2022-09-30,3,-0.285740,-0.388110,-0.245859,-0.315915,2.005748,-0.126131,-0.466580


In [35]:
from sklearn.ensemble import GradientBoostingRegressor

# Training data
X = num_viajes_weather.loc[:, ['cluster', 'hour', 'dia_semana', 'mes',
       'tmin', 'pres']]  # features
y = num_viajes_weather.loc[:, 'trips']  # target

# Test data
Z = test.loc[:, ['cluster', 'hour', 'dia_semana', 'mes',
       'tmin', 'pres']]


model = GradientBoostingRegressor(loss="absolute_error", n_estimators=100, learning_rate=0.1, max_depth=10)

model.fit(X, y)


pred = pd.Series(model.predict(Z), index=Z.index)

In [36]:
test['prediction']= pred
test['prediction'] = test['prediction'].fillna(0)
test.tail()

,Id,cluster,date,hour,dia_semana,mes,season,tavg,tmin,tmax,prcp,wdir,wspd,pres,prediction
131833,45_2023-01-01_00,45,2023-01-01,0,6,1,4,-1.117715,-1.056661,-0.968687,-0.341636,-0.39794,-1.526017,1.395076,1.433505
131834,59_2023-01-01_00,59,2023-01-01,0,6,1,4,-1.117715,-1.056661,-0.968687,-0.341636,-0.39794,-1.526017,1.395076,1.671002
131835,25_2023-01-01_00,25,2023-01-01,0,6,1,4,-1.117715,-1.056661,-0.968687,-0.341636,-0.39794,-1.526017,1.395076,2.021200
131836,37_2023-01-01_00,37,2023-01-01,0,6,1,4,-1.117715,-1.056661,-0.968687,-0.341636,-0.39794,-1.526017,1.395076,1.708756
131837,85_2023-01-01_00,85,2023-01-01,0,6,1,4,-1.117715,-1.056661,-0.968687,-0.341636,-0.39794,-1.526017,1.395076,2.324082


**Preparing kaggle format**

In [37]:
test.tail()

,Id,cluster,date,hour,dia_semana,mes,season,tavg,tmin,tmax,prcp,wdir,wspd,pres,prediction
131833,45_2023-01-01_00,45,2023-01-01,0,6,1,4,-1.117715,-1.056661,-0.968687,-0.341636,-0.39794,-1.526017,1.395076,1.433505
131834,59_2023-01-01_00,59,2023-01-01,0,6,1,4,-1.117715,-1.056661,-0.968687,-0.341636,-0.39794,-1.526017,1.395076,1.671002
131835,25_2023-01-01_00,25,2023-01-01,0,6,1,4,-1.117715,-1.056661,-0.968687,-0.341636,-0.39794,-1.526017,1.395076,2.021200
131836,37_2023-01-01_00,37,2023-01-01,0,6,1,4,-1.117715,-1.056661,-0.968687,-0.341636,-0.39794,-1.526017,1.395076,1.708756
131837,85_2023-01-01_00,85,2023-01-01,0,6,1,4,-1.117715,-1.056661,-0.968687,-0.341636,-0.39794,-1.526017,1.395076,2.324082


In [38]:
test.rename(columns={'prediction': 'trips'}, inplace=True)
test = pd.DataFrame(data=test, columns=["Id", "trips"])
test

,Id,trips
0,72_2022-09-30_22,3.130711
1,93_2022-09-30_22,2.134862
2,36_2022-09-30_22,2.779451
3,83_2022-09-30_22,1.359704
4,92_2022-09-30_22,2.704060
...,...,...
131833,45_2023-01-01_00,1.433505
131834,59_2023-01-01_00,1.671002
131835,25_2023-01-01_00,2.021200
131836,37_2023-01-01_00,1.708756


In [39]:
test.dtypes

Id        object
trips    float64
dtype: object

In [40]:
test.to_csv('SearchGV.csv',index=False)